In [139]:
from flask import Flask, request, jsonify
from neo4j import GraphDatabase
import nest_asyncio
import threading

nest_asyncio.apply()

In [140]:
URI = "neo4j+s://398500ef.databases.neo4j.io"
AUTH = ("neo4j", "l_1gCVeywnEgHdmLxf93BdT7kp01G30o5ZpTORaI-y4")

In [141]:
driver = GraphDatabase.driver(URI, auth=(AUTH))

In [142]:
def run_query(query, parameters=None):
    with driver.session() as session:
        results = session.run(query, parameters)
        return [dict(record) for record in results]

In [143]:
def create_user(name, age, location, interests):
    try:
        query = """
                CREATE (u:User {name: $name, age: $age, location: $location, interests: $interests})
                RETURN u
                """
        parameters = {"name": name, "age": age, "location":location, "interests":interests}
        run_query(query, parameters)
    except Exception as e:
        raise RuntimeError(f"Error creating friendship: {str(e)}")

In [144]:
def update_user(name, **kwargs):
    try:
        set = ",".join([f"u.{key} = ${key}" for key in kwargs])
        query = ("MATCH (u:User {name: $name}) "
                f"SET {set} "
                "RETURN u")
                
        parameters = {"name": name, **kwargs}
        run_query(query, parameters)
    except Exception as e:
        raise RuntimeError(f"Error updating user: {str(e)}")
    

In [145]:
def create_friendship(user1, user2):
    query = """
            MATCH (u1:User {name: $user1}),
            (u2:User {name: $user2})
            CREATE (u1)-[:FRIENDS_WITH]->(u2), (u2)-[:FRIENDS_WITH]->(u1)
            RETURN u1, u2
            """
    parameters = {"user1": user1, "user2": user2}
    run_query(query, parameters)
   


In [146]:
def remove_friendship(user1, user2):
    query = """
            MATCH (u1:User {name: $user1})-[r:FRIENDS_WITH]-(u2:User {name: $user2})
            DELETE r
            """
    parameters = {"user1": user1, "user2": user2}
    result = run_query(query, parameters)
    print(result)

In [147]:
def send_friend_request(user1, user2):
    query = """
            MATCH (u1:User {name: $user1}),
            (u2:User {name: $user2})
            CREATE (u1)-[:OUTGOING_REQUEST]->(u2), (u2)-[:INCOMING_REQUEST]->(u1)
            RETURN u1, u2
            """
    parameters = {"user1": user1, "user2": user2}
    result = run_query(query, parameters)
    print(result)

In [148]:
def respond_to_friend_request(user1, user2, accept):
    query = (
            "MATCH (u1:User {name: $from_user})-[r1:OUTGOING_REQUEST]->(u2:User {name: $to_user}) "
            "MATCH (u2)-[r2:INCOMING_REQUEST]->(u1) "
            "DELETE r1, r2 "
    )
    if accept:
        query += "CREATE (u1)-[:FRIENDS_WITH]->(u2), (u2)-[:FRIENDS_WITH]->(u1)"
    
    parameters = {"from_user": user1, "to_user": user2}
    result = run_query(query, parameters)
    print(result)

In [175]:
def post(user, post):
    query = """
            MATCH (u:User {name: $user})
            CREATE (u)-[:POSTED]->(p:Post {content: $post, no_of_likes: 0, comments: []})
            RETURN p.id
            """
    parameters = {"user": user, "post": post}
    result = run_query(query, parameters)
    print(result)

In [183]:
def like_post(user, post_id):
    query = """
            MATCH (u:User {name: $user})
            MATCH (p:Post) WHERE elementId(p) = $post
            SET p.no_of_likes = coalesce(p.no_of_likes, 0) + 1
            CREATE (u)-[:LIKED]->(p)
            RETURN p
            """
    parameters = {"user": user, "post": post_id}
    result = run_query(query, parameters)
    print(result)

In [151]:
def create_group(name):
    query = """
            CREATE (g:Group {name: $name})
            RETURN g
            """
    parameters = {"name": name}
    result = run_query(query, parameters)
    print(result)

In [152]:
def join_group(group, user):
    query = """
            MATCH (g:Group {name: $name})
            MATCH (u:User {name: $user})
            CREATE (u)-[:MEMBER_OF]->(g)
            RETURN u, g
            """
    parameters = {"name": group, "user": user}
    result = run_query(query, parameters)
    print(result)

In [185]:
def comment_on_post(user, post, comment):
    query = """
            MATCH (u:User {name: $user})
            MATCH (p:Post) WHERE elementId(p) = $post
            CREATE (u) -[:COMMENTED{comment: $comment}]-> (p)
            SET p.comments = coalesce(p.comments, []) + [$comment]
            RETURN p
            """
    parameters = {"user": user, "post": post, "comment": comment }
    result = run_query(query, parameters)
    print(result)

In [154]:
def friend_recommendation(user):
    query = """
            MATCH (u:User{name:$user})-[:FRIENDS_WITH] -> (u2) <-[:FRIENDS_WITH]- (ru)
            WHERE NOT (u)-[:FRIENDS_WITH] -> (ru) and u <> ru
            RETURN ru
            """
    parameters = {"user": user}
    result = run_query(query, parameters)
    print(result)

In [155]:
def search_user(**kwargs):
    where = ",".join([f"u.{key} = ${key}" for key in kwargs])
    query = (
        "MATCH (u:User) "
        f"WHERE {where} "
        "RETURN u"
    )

    parameters = {**kwargs}
    result = run_query(query,parameters)
    print(result)

In [156]:
app = Flask(__name__)

In [157]:
@app.route('/register', methods=['POST'])
def register_user():
    data = request.json
    create_user(data['name'], data['age'], data['location'], data['interests'])
    return jsonify({'status': 'User registered'})


In [158]:
@app.route('/update', methods=['POST'])
def update_user_info():
    data = request.json
    update_user(data['name'], **data['update'])
    return jsonify({'status': 'User information updated'})

In [159]:
@app.route('/friends', methods=['POST'])
def create_friendship_handler():
    data = request.json
    create_friendship(data['user1'], data['user2'])
    return jsonify({'status': 'Friendship created'})

In [160]:
@app.route('/friends', methods=['DELETE'])
def remove_friendship_handler():
    data = request.json
    remove_friendship(data['user1'], data['user2'])
    return jsonify({'status': 'Friendship removed'})

In [161]:
@app.route('/sendrequest', methods=['POST'])
def send_friend_request_handler():
    data = request.json
    send_friend_request(data["user1"], data["user2"])
    return jsonify({'status': 'Friend request sent'})

In [162]:
@app.route('/respondrequest', methods=['POST'])
def respond_friend_request_handler():
    data = request.json
    respond_to_friend_request(data["user1"], data["user2"], data["accept"])
    return jsonify({'status': 'responded to friend request '})

In [163]:
@app.route('/post', methods=['POST'])
def post_handler():
    data = request.json
    post(data["user"], data["post"])
    return jsonify({'status': 'post created'})

In [164]:
@app.route('/post/like', methods=['POST'])
def like_post_handler():
    data = request.json
    like_post(data["user"], data["post"])
    return jsonify({'status': 'post liked'})
    

In [165]:
@app.route('/group/create', methods=['POST'])
def create_group_handler():
    data = request.json
    create_group(data["name"])
    return jsonify({'status': 'group created'})

In [166]:
@app.route('/group/join', methods=['POST'])
def join_group_handler():
    data = request.json
    join_group(data["group"], data["user"])
    return jsonify({'status': 'user joined group'})

In [167]:
@app.route('/post/comment', methods=['POST'])
def comment_on_post_handler():
    data = request.json
    comment_on_post(data["user"], data["post"], data["comment"])
    return jsonify({'status': 'commented on post'})

In [168]:
@app.route('/friend/recommendation', methods=['POST'])
def friend_recommendation_handler():
    data = request.json
    friend_recommendation(data["user"])
    return jsonify({'status': 'recommended friends'})

In [169]:
@app.route('/search', methods=['POST'])
def create_friendship():
    data = request.json
    search_user(**data)
    return jsonify({'status': 'searched user'})

In [ ]:
def run_app():
    app.run(port=8080)

thread = threading.Thread(target=run_app)
thread.start()

 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit


In [177]:
import requests

response = requests.post('http://localhost:8080/register', json={
    'name': 'john',
    'age': 21,
    'location': 'Ethiopia',
    'interests': ['reading', 'hiking']
})
print(response.json())

{'status': 'User registered'}


In [ ]:
response = requests.post('http://localhost:8080/update', json={
    'name': 'liel',
    'update': {
        'age': 50,
        'location': 'San Francisco'
    }
})
print(response.json())

{'status': 'User information updated'}


In [ ]:
response = requests.post('http://localhost:8080/friends', json={
    'user1': 'Chappie',
    'user2': 'Charlie'
})
print(response.json())

127.0.0.1 - - [05/Jun/2024 09:49:39] "POST /friends HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2024 09:50:29] "POST /sendrequest HTTP/1.1" 200 -


{'status': 'Friendship created'}


In [ ]:
response = requests.delete('http://localhost:8080/friends', json={
    'user1': 'Chappie',
    'user2': 'Charlie'
})
print(response.json())

[]


{'status': 'Friendship removed'}


In [ ]:
response = requests.post('http://localhost:8080/sendrequest', json={
    'user1': 'liel',
    'user2': 'Charlie'
})
print(response.json())

127.0.0.1 - - [05/Jun/2024 09:54:41] "POST /sendrequest HTTP/1.1" 200 -


[{'u1': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:95' labels=frozenset({'User'}) properties={'name': 'liel', 'location': 'San Francisco', 'interests': ['reading', 'hiking'], 'age': 50}>, 'u2': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:42' labels=frozenset({'User'}) properties={'name': 'Charlie', 'location': 'New York', 'interests': ['Music', 'Running'], 'age': 22}>}]


{'status': 'Friend request sent'}


In [ ]:
response = requests.post('http://localhost:8080/respondrequest', json={
    'user1': 'liel',
    'user2': 'Charlie',
    'accept': True
})
print(response.json())

127.0.0.1 - - [05/Jun/2024 09:55:08] "POST /respondrequest HTTP/1.1" 200 -


[]


{'status': 'responded to friend request '}


In [178]:
response = requests.post('http://localhost:8080/post', json={
    'user': 'Charlie',
    'post': 'this is a test post'
})
print(response.json())

{'status': 'post created'}


In [184]:
response = requests.post('http://localhost:8080/post/like', json={
    'user': 'Alice',
    'post': '4:73dec725-ccdf-437b-bfd5-623161b3662c:118'
})
print(response.json())

{'status': 'post liked'}


In [187]:
response = requests.post('http://localhost:8080/post/comment', json={
    'user': 'liel',
    'post': '4:73dec725-ccdf-437b-bfd5-623161b3662c:118',
    'comment': 'cool post'
})
print(response.json())

{'status': 'commented on post'}


In [ ]:
response = requests.post('http://localhost:8080/group/create', json={
    'name': 'interesting'
})
print(response.json())

127.0.0.1 - - [05/Jun/2024 10:07:41] "POST /group/create HTTP/1.1" 200 -


[{'g': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:104' labels=frozenset({'Group'}) properties={'name': 'interesting'}>}]


{'status': 'group created'}


In [ ]:
response = requests.post('http://localhost:8080/group/join', json={
    'group': 'interesting',
    'user': 'Basliel'
})
print(response.json())

127.0.0.1 - - [05/Jun/2024 10:09:46] "POST /group/join HTTP/1.1" 200 -


[{'u': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:84' labels=frozenset({'User'}) properties={'name': 'Basliel', 'location': 'Ethiopia', 'interests': ['reading', 'hiking'], 'age': 21}>, 'g': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:104' labels=frozenset({'Group'}) properties={'name': 'interesting'}>}]


{'status': 'user joined group'}


In [ ]:
response = requests.post('http://localhost:8080/friend/recommendation', json={
    'user': 'Bob',
})
print(response.json())

127.0.0.1 - - [05/Jun/2024 10:12:54] "POST /friend/recommendation HTTP/1.1" 200 -


[{'ru': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:75' labels=frozenset({'User'}) properties={'name': 'Chappie', 'location': 'Ethiopia', 'interests': ['reading', 'hiking'], 'age': 21}>}, {'ru': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:95' labels=frozenset({'User'}) properties={'name': 'liel', 'location': 'San Francisco', 'interests': ['reading', 'hiking'], 'age': 50}>}]


{'status': 'recommended friends'}


In [ ]:
response = requests.post('http://localhost:8080/search', json={
    'age': 41,
})
print(response.json())

127.0.0.1 - - [05/Jun/2024 10:13:59] "POST /search HTTP/1.1" 200 -


[{'u': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:19' labels=frozenset({'User'}) properties={'name': 'Alice', 'location': 'San Francisco', 'interests': ['Music', 'Running'], 'age': 41}>}]


{'status': 'searched user'}
